In [0]:
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,DateType
from pyspark.sql.functions import col,when,sum
from datetime import datetime

spark = SparkSession.builder.appName("app").master("local[2]").getOrCreate()

In [0]:
 schema = StructType([
     StructField("emp_id",IntegerType(),False),
     StructField("event_day",DateType(),False),
     StructField("in_time",IntegerType(),False),
     StructField("out_time",IntegerType(),False)
 ])
 
data = [
( 1      , datetime(2020,11,28) , 4       , 32  )     ,
( 1      , datetime(2020,11,28) , 55      , 200 )     ,
( 1      , datetime(2020,12,3 ) , 1       , 42  )     ,
( 2      , datetime(2020,11,28) , 3       , 33  )     ,
( 2      , datetime(2020,12,9 ) , 47      , 74  )     
]
emp = spark.createDataFrame(data,schema)
emp.show()

+------+----------+-------+--------+
|emp_id| event_day|in_time|out_time|
+------+----------+-------+--------+
|     1|2020-11-28|      4|      32|
|     1|2020-11-28|     55|     200|
|     1|2020-12-03|      1|      42|
|     2|2020-11-28|      3|      33|
|     2|2020-12-09|     47|      74|
+------+----------+-------+--------+



In [0]:
# Write a solution to calculate the total time in minutes spent by each employee on each day at the office. Note that within one day, an employee can enter and leave more than once. 
# The time spent in the office for a single entry is out_time - in_time. Return the result table in any order.
emp.groupBy("event_day","emp_id").agg(sum(col("out_time")-col("in_time")).alias('total_time')).show()

+----------+------+----------+
| event_day|emp_id|total_time|
+----------+------+----------+
|2020-11-28|     1|       173|
|2020-12-03|     1|        41|
|2020-11-28|     2|        30|
|2020-12-09|     2|        27|
+----------+------+----------+



In [0]:
emp.createOrReplaceTempView("emp")
spark.sql("select event_day,emp_id,sum(out_time-in_time) total_time from emp group by 1,2").show()

+----------+------+----------+
| event_day|emp_id|total_time|
+----------+------+----------+
|2020-11-28|     1|       173|
|2020-12-03|     1|        41|
|2020-11-28|     2|        30|
|2020-12-09|     2|        27|
+----------+------+----------+



In [0]:
spark.stop()